In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import pickle

plt.rc('font', family='Apple SD Gothic Neo')

### 구글 '제주특별자치도' 즐길거리 top 47 데이터 불러오기

In [2]:
for num in range(1,6):
    with open('./data/pandemic_'+str(num)+'.pkl', 'rb') as f:
            globals()['pandemic_{}'.format(str(num))] = pickle.load(f)
            
with open('./data/extra_data.pkl', 'rb') as f:
    extra_data = pickle.load(f)

combined_df = pd.concat([pandemic_1, pandemic_2, pandemic_3, pandemic_4, pandemic_5, extra_data])
combined_df.drop('jeju_place', axis=1, inplace=True)
combined_df['날짜'] = pd.to_datetime(combined_df['날짜'])
combined_df = combined_df.sort_values(by='날짜', ascending=True).reset_index(drop=True)
combined_df.head()

,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place_google
0,2020-01-01 00:00:00,"제주여행의 시작 : 국립제주박물관 (제주 가볼만한곳, 제주 유배인의 이야기, 비 오...",제주여행의 시작\n국립제주박물관\n박물관 도장깨기!이번 저의 제주도 여행 테마는 ‘...,https://blog.naver.com/tnrwlschl/221753997486,제주여행의 시작 국립제주박물관 박물관 도장깨기 이번 저의 제주도 여행 테마는 다크...,"[제주, 여행의, 시작, 국립제주박물관, 박물관, 도장깨기, 이번, 저의, 제주, ...",[]
1,2020-01-01 01:15:00,"2019 제주여행 3 :: 성산일출봉, 풍미 독서, 세화해수욕장, 모이소",2박 3일 제주여행의 마지막 날 아침\n성산 일출봉을 향했다.입장 시간이 이미 해가...,https://blog.naver.com/chersy/221755888807,박 일 제주여행의 마지막 날 아침 성산 일출봉을 향했다.입장 시간이 이미 해가 ...,"[제주, 여행의, 마지막, 아침, 성산, 일출봉을, 향했다., 입장, 시간, 이미,...",[성산일출봉]
2,2020-01-01 01:49:00,[제주여행] 제주 신라스테이 숙박 후기,2019.11.20 - 2019.11.26\n제주에서의 마지막 밤은회사 복지포인트 ...,https://blog.naver.com/pipi4692/221755906364,. . . . 제주에서의 마지막 밤은회사 복지포인트 찬...,"[제주, 에서의, 마지막, 밤은회사, 복지포인트, 찬스로, 제주, 신라스테이, 에서...",[]
3,2020-01-01 02:03:00,[제주여행] 김포공항& 제주함덕해수욕장 &제주 무거버거♡,안녕하세요ㅎ\n바나나맛우유입니다ㅎㅎ\n앞전에 호텔사진 보고 오신분은\n알겠지만ㅋㅋㅋ...,https://blog.naver.com/rnjsrn001/221755913575,안녕하세요 바나나맛우유입니다 앞전에 호텔사진 보고 오신분은 알겠지만 네 ...,"[안녕하세요, 바나나맛우유, 입니다, 앞전에, 호텔, 사진, 보고, 오신분은, 알겠...",[함덕해수욕장]
4,2020-01-01 04:13:00,제주여행 2박3일 필수코스! 애월한담공원에서 산책하기 :> (카약은 덤~),제주여행 2박3일 필수코스! 애월한담공원에서 산책하기 :> (카약은 덤~)한달전에 ...,https://blog.naver.com/younkuen12/221755964865,제주여행 박 일 필수코스 애월한담공원에서 산책하기 카약은 덤 한달전에 ...,"[제주, 여행, 필수, 코스, 애월, 한담공원에서, 산책, 하기, 카약은, 한달전에...",[]


In [167]:
final_df = combined_df[combined_df['jeju_place_google'].apply(lambda x: len(x)!=0)]
final_df.reset_index(drop=True,inplace=True)
print(len(final_df))
final_df.head()

29795


,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place_google
0,2020-01-01 01:15:00,"2019 제주여행 3 :: 성산일출봉, 풍미 독서, 세화해수욕장, 모이소",2박 3일 제주여행의 마지막 날 아침\n성산 일출봉을 향했다.입장 시간이 이미 해가...,https://blog.naver.com/chersy/221755888807,박 일 제주여행의 마지막 날 아침 성산 일출봉을 향했다.입장 시간이 이미 해가 ...,"[제주, 여행의, 마지막, 아침, 성산, 일출봉을, 향했다., 입장, 시간, 이미,...",[성산일출봉]
1,2020-01-01 02:03:00,[제주여행] 김포공항& 제주함덕해수욕장 &제주 무거버거♡,안녕하세요ㅎ\n바나나맛우유입니다ㅎㅎ\n앞전에 호텔사진 보고 오신분은\n알겠지만ㅋㅋㅋ...,https://blog.naver.com/rnjsrn001/221755913575,안녕하세요 바나나맛우유입니다 앞전에 호텔사진 보고 오신분은 알겠지만 네 ...,"[안녕하세요, 바나나맛우유, 입니다, 앞전에, 호텔, 사진, 보고, 오신분은, 알겠...",[함덕해수욕장]
2,2020-01-01 05:33:00,2019 돌아보며(ft.제주 여행 교훈),1. 부지런해져라. \n-일찍 자고 일찍 일어나기 (10-04)\n-생각한 대로 살...,https://blog.naver.com/umyumybabies/221755987867,. 부지런해져라. 일찍 자고 일찍 일어나기 생각한 대로 살자....,"[부지런, 해져라., 일찍, 자고, 일찍, 일어나, 생각, 대로, 살자., 살아가는...",[비양도]
3,2020-01-01 10:46:00,"제주여행 3일차(용두암,전복성게미역국, 선물고팡,제주공항)",올것이 왔습니다. 제주에서의 마지막 날입니다.\n너무 아쉽습니다.\n혼자여행 외롭거...,https://blog.naver.com/deuxsun0915/221756111749,올것이 왔습니다. 제주에서의 마지막 날입니다. 너무 아쉽습니다. 혼자여행 외롭거나 ...,"[올것이, 왔습니다., 제주, 에서의, 마지막, 날입니다., 너무, 아쉽습니다., ...",[용두암]
4,2020-01-01 10:51:00,2019.12.31. 제주여행 2일차(먹고자고),오늘은 넘나 심플한 하루였다. 늦잠자고 점심먹고 스벅갔다가 저녁먹고 끝!원래는 10...,https://blog.naver.com/hongnal0127/221756115426,오늘은 넘나 심플한 하루였다. 늦잠자고 점심먹고 스벅갔다가 저녁먹고 끝 원래는 ...,"[오늘은, 넘나, 심플한, 하루였다., 늦잠자고, 점심, 먹고, 스벅갔다가, 저녁,...",[한라산]


In [ ]:
# pickel 저장
with open('./data/jeju_place_top47.pkl', 'wb') as f:
    pickle.dump(final_df, f, pickle.HIGHEST_PROTOCOL)

In [2]:
# pickel 불러오기
with open('./data/jeju_place_top47.pkl', 'rb') as f:
    jeju_top47 = pickle.load(f)
    
jeju_top47.head()

,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place_google
0,2020-01-01 01:15:00,"2019 제주여행 3 :: 성산일출봉, 풍미 독서, 세화해수욕장, 모이소",2박 3일 제주여행의 마지막 날 아침\n성산 일출봉을 향했다.입장 시간이 이미 해가...,https://blog.naver.com/chersy/221755888807,박 일 제주여행의 마지막 날 아침 성산 일출봉을 향했다.입장 시간이 이미 해가 ...,"[제주, 여행의, 마지막, 아침, 성산, 일출봉을, 향했다., 입장, 시간, 이미,...",[성산일출봉]
1,2020-01-01 02:03:00,[제주여행] 김포공항& 제주함덕해수욕장 &제주 무거버거♡,안녕하세요ㅎ\n바나나맛우유입니다ㅎㅎ\n앞전에 호텔사진 보고 오신분은\n알겠지만ㅋㅋㅋ...,https://blog.naver.com/rnjsrn001/221755913575,안녕하세요 바나나맛우유입니다 앞전에 호텔사진 보고 오신분은 알겠지만 네 ...,"[안녕하세요, 바나나맛우유, 입니다, 앞전에, 호텔, 사진, 보고, 오신분은, 알겠...",[함덕해수욕장]
2,2020-01-01 05:33:00,2019 돌아보며(ft.제주 여행 교훈),1. 부지런해져라. \n-일찍 자고 일찍 일어나기 (10-04)\n-생각한 대로 살...,https://blog.naver.com/umyumybabies/221755987867,. 부지런해져라. 일찍 자고 일찍 일어나기 생각한 대로 살자....,"[부지런, 해져라., 일찍, 자고, 일찍, 일어나, 생각, 대로, 살자., 살아가는...",[비양도]
3,2020-01-01 10:46:00,"제주여행 3일차(용두암,전복성게미역국, 선물고팡,제주공항)",올것이 왔습니다. 제주에서의 마지막 날입니다.\n너무 아쉽습니다.\n혼자여행 외롭거...,https://blog.naver.com/deuxsun0915/221756111749,올것이 왔습니다. 제주에서의 마지막 날입니다. 너무 아쉽습니다. 혼자여행 외롭거나 ...,"[올것이, 왔습니다., 제주, 에서의, 마지막, 날입니다., 너무, 아쉽습니다., ...",[용두암]
4,2020-01-01 10:51:00,2019.12.31. 제주여행 2일차(먹고자고),오늘은 넘나 심플한 하루였다. 늦잠자고 점심먹고 스벅갔다가 저녁먹고 끝!원래는 10...,https://blog.naver.com/hongnal0127/221756115426,오늘은 넘나 심플한 하루였다. 늦잠자고 점심먹고 스벅갔다가 저녁먹고 끝 원래는 ...,"[오늘은, 넘나, 심플한, 하루였다., 늦잠자고, 점심, 먹고, 스벅갔다가, 저녁,...",[한라산]


## 관광지별 문장 split 후 관련 속성 뽑아내기

In [3]:
# 분석 날짜: 2021년 1월 ~ 2021년 6월
jeju_top47 = jeju_top47[(jeju_top47['날짜'].dt.year==2021) & (jeju_top47['날짜'].dt.month < 7)]
jeju_top47.reset_index(drop=True, inplace=True)
jeju_top47.head()

,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place_google
0,2021-01-01 03:32:00,제주 오름 추천 ㅣ 금오름 소요시간 ♥,미니 백록담이라고도 불리는 제주 금오름 ☻ \n보통 언덕인 제주 오름들과는 다르게 ...,https://blog.naver.com/jinmichu/222192225676,미니 백록담이라고도 불리는 제주 금오름 보통 언덕인 제주 오름들과는 다르게 ...,"[미니, 백록담, 이라고도, 불리는, 제주, 금오름, 보통, 언덕, 제주, 오름들과...",[한라산]
1,2021-01-01 07:03:00,2021 새해 첫 해외여행은 미국 서부로!,"""해가 밝았으니 여행을 떠나자!""\n신축년 새해는 소처럼 우직하니 해외여행\n첫 스...",https://blog.naver.com/l6014417/222192276207,해가 밝았으니 여행을 떠나자 신축년 새해는 소처럼 우직하니 해외여행 첫 스타트...,"[해가, 밝았, 으니, 여행, 떠나, 신축년, 새해는, 소처럼, 우직하니, 해외여행...",[성산일출봉]
2,2021-01-01 07:19:00,"제주 여행 - 제주 한라산 등반 - 아들과 함께하는 한라산, 어승생악과 윗세오름을 오르다",우리 아들 5살때 둘째 태어난지 50일경 즈음..\n둘째 스트레스를 자신이 앓고 있...,https://blog.naver.com/loverboy222/222192280390,우리 아들 살때 둘째 태어난지 일경 즈음.. 둘째 스트레스를 자신이 앓고 있어...,"[우리, 아들, 살때, 둘째, 태어난, 일경, 즈음, .., 둘째, 스트레스, 자신...",[한라산]
3,2021-01-01 07:35:00,12/31/2020 제주여행 2,작년 결혼기념일(31주년)기념으로 같이 산 롱 패딩에\n32주년 기념 모자는 이번 ...,https://blog.naver.com/jym64/222192284434,작년 결혼기념일 주년 기념으로 같이 산 롱 패딩에 주년 기념 모자는 이번 여...,"[작년, 결혼기념일, 주년, 기념으로, 같이, 패딩, 주년, 기념, 모자는, 이번,...","[중문관광단지, 산방산, 색달해수욕장, 용머리해안, 송악산]"
4,2021-01-01 08:18:00,"나홀로 여행 :-) 3일차, 후회없는 제주 마지막날",2020.11.12.목\n나의 마지막 제주 일정을ㅋㅋ\n미루다 새해에여튼 시작\n<...,https://blog.naver.com/ehsdhs/222192297761,. . .목 나의 마지막 제주 일정을 미루다 새해에여튼 시작 숙소 ...,"[.목, 나의, 마지막, 제주, 일정을, 미루다, 새해에, 여튼, 시작, 숙소, 정...",[돌문화공원]


In [4]:
# jeju_place_google 칼럼의 리스트를 돌면서 다음 관광지가 나오기 전까지의 문장들을 새 dataframe에 집어넣기
# => 하나의 관광지에 대한 관련 문장들을 뽑아내기 위해

# for places in jeju_top47['jeju_place_google']:
#     if '성산일출봉' in places:
#         places.append('성산 일출봉')
        
place_len = 0
for i in jeju_top47['jeju_place_google']:
    place_len += len(i)
    
new_df = pd.DataFrame(index=range(0,place_len), columns=['날짜', 'text', '관광지', 'index'])
new_df.head(1)

,날짜,text,관광지,index
0,NaN,NaN,NaN,NaN


In [5]:
df_copy = jeju_top47.copy()

In [18]:
for text in df_copy['text_prep'][0].split('.'):
    for place in df_copy['jeju_place_google'][0]:
        if place in text:
            print(text)

   금오름 정상 도착 완료   이때까지만 해도   월이지만 날이 더웠어서  땀흘리면서 올라갔네요      금오름 물이 조금밖에 없어서 슬픈 진미츄      제주 여행에서 한라산 백록담  금오름 정상 모두  물이 말라있었음              고려 못한 복장도 약간 에러      그래도 신발은 워커라 아닌 중 다행이였어요      금오름 정상이 흙  잔디로 되어있어서  아끼는 신발  흰색 운동화 등    신고 가면  후회하실거예여 이 금오름 정상에서 내려다보면  웅장함에 우와  소리가 절로 나는데   사진과 영상에서는 금오름의 스케일이  잘 담기지 않은 것 같아요     아래로 내려가면 또 그만큼 올라가야하기에  내려갈지말지 고민하다가       그래도 이왕 와본거 내려가서 반대쪽으로 올라갔어요 우와앙 반대편쪽으로 걸어가다보니  멋진 말 두마리도 볼 수 있었음   말과 함께 인증샷 찍으시는 분들도 꽤 많던데  저는 쫄보라 멀리서 구경만    반대편으로 올라온 모습       분화구에 물이 말라있어서 아쉬움이 남아요  푸릇푸릇한 봄 여름에  다시 한 번 가봐야겠어요   그리구 제주 금오름 언덕이  패러글라이딩 맛집   인가봐요  체험이 아니라 취미로 하시는 분들이  꽤 많더라고요    자유자재로 비행하시는 모습이 멋있어서  한참을 바라보다 내려왔답니당 패러글라이딩 구경하는 재미도 쏠쏠해요  다들 그냥 지나치지 못하고  우와   우와   연발하며 구경잼        금오름 정상에서 본 제주 서부지역    금오름은 금악마을의 중심에 위치해있어서  제주 서쪽 전부를 내려다볼 수 있다고 하네용  제주 대표 오름 중 하나인 제주 금오름  제주 여행 중 꼭 들려야 할 명소 인정입니다 제주 오름 추천  새별오름 포스팅 링크도  첨부하니 참고하세용     제주오름추천  제주여행  제주도여행  제주금오름  금오름 


In [100]:
text_list = ''
for text in jeju_top47['text_prep'][0].split('.'):
    text_list = text_list + text + '.'
    
text_list

' 박  일 제주여행의 마지막 날 아침 성산 일출봉을 향했다.입장 시간이 이미 해가 다 뜬 시간이라 일출을 보지는 못했지만 한번 올라가 보기로 했다. 해가 너무 비춰서 역광이.. 사진 찍기는 좀 힘들었다.그래도 올라가며 내려다보며 찍는 사진은 좋 다   월   일의 제주도는 생각보다 더웠는데 우린 열심히 땀 흘리며 올라 정상을 보았다.정상에서 보니 바다가 또 이렇게 푸르르다 성산일출봉 등반 완료 성산 일출봉을 오르고 나서 배가 많이 고팠던 우린 바로 브런치를 먹기로 했다.우리가 첫 손님으로 가서  자리 선택이 자유로웠다 창가 자리가 탐났는데.. 주문을 많이 해서 테이블이 작을 거라고 ㅠ.ㅠ 옮겼다.근데 음식이 나오니 큰 테이블 이용하길 잘했다.와일드 통새우 버거 정말 좋았다   새우 사랑   다른 음식들도 맛있었는데  그래도 나는 새우가 제일 좋았다. 개취 존중   든든하게 한 끼를 또 먹고  바다를 마지막으로 보고 돌아가기로 했다.바다에 오니 또 날씨가 우중충해지고   바다랑 우리 안 맞나 바람에 머리가 제멋대로 날렸지만  바다니까  기분이 좋았던 것 같다. 사진을 무지 찍은 걸 보면   렌트 반납하고 공항 가기 전  기념품을 몇 가지 볼까 해서 들린  모이소  층은 차를 마실 수 있는 공간이 있고  층이 기념품들이 모여있다.우리는 차를 좋아해서  레드향 차를 하나 구입해 왔다. 층 기념품들 중에서는 집에 둘 만한 방향제와 제주느낌 주는 자석 선택 모이소에서 구입한 건  바다 느낌의 자석  돌하르방 방향제  레드향차  자석은 냉장고 옆면에 잘 있고  방향제는 현재 잘 사용중이고 차한잔의 여유를 좋아하는 우리는 레드향차는 다 먹었다..'

In [6]:
df_test = df_copy.loc[[3]]
df_test

,날짜,제목,내용,url,text_prep,tokenized_text,jeju_place_google
3,2021-01-01 07:35:00,12/31/2020 제주여행 2,작년 결혼기념일(31주년)기념으로 같이 산 롱 패딩에\n32주년 기념 모자는 이번 ...,https://blog.naver.com/jym64/222192284434,작년 결혼기념일 주년 기념으로 같이 산 롱 패딩에 주년 기념 모자는 이번 여...,"[작년, 결혼기념일, 주년, 기념으로, 같이, 패딩, 주년, 기념, 모자는, 이번,...","[중문관광단지, 산방산, 색달해수욕장, 용머리해안, 송악산]"


In [7]:
new_df = pd.DataFrame(index=range(0,place_len), columns=['날짜', 'text', '관광지', 'index'])
new_df.head(1)

,날짜,text,관광지,index
0,NaN,NaN,NaN,NaN


In [11]:
new_df_idx = 0

for idx in range(len(df_test)):
    # 한 행마다 돌면서 관광지에 대한 문장이 있을 경우 str에 추가하여 new_df 관광지 컬럼에 추가
    places = df_test.iloc[idx]['jeju_place_google']
    i=0 # 관광지 리스트의 index
    place_texts = ''
    for text in df_test.iloc[idx]['내용'].split('\n'):
        if i == len(places) - 1: #관광지가 1개인 경우 or 맨 마지막 관광지인 경우
            place_texts = place_texts + text + '.'
        else:
            if places[i] in text:
                place_texts = place_texts + text + '.'
                if places[i+1] in text: #같은 문장에 다음 관광지도 포함된다면
                    new_df.loc[new_df_idx + i]['날짜'] = df_test.iloc[idx]['날짜']
                    new_df.loc[new_df_idx + i]['text'] = place_texts
                    new_df.loc[new_df_idx + i]['index'] = idx
                    new_df.loc[new_df_idx + i]['관광지'] = places[i]
                    print(places[i],':', place_texts, '\n')
                    i = i +1
                    
                    new_df.loc[new_df_idx + i]['날짜'] = df_test.iloc[idx]['날짜']
                    new_df.loc[new_df_idx + i]['text'] = place_texts
                    new_df.loc[new_df_idx + i]['index'] = idx
                    new_df.loc[new_df_idx + i]['관광지'] = places[i]
                    print(places[i],':', place_texts, '\n')
                    place_texts = ''

            elif places[i+1] in text: #현재 문장에서 다음 관광지가 나올 경우, 누적된 text를 현재 관광지 칼럼에 저장
                new_df.loc[new_df_idx + i]['날짜'] = df_test.iloc[idx]['날짜']
                new_df.loc[new_df_idx + i]['text'] = place_texts
                new_df.loc[new_df_idx + i]['index'] = idx
                new_df.loc[new_df_idx + i]['관광지'] = places[i]
                i += 1
                place_texts = ''
                place_texts = place_texts + text + '.' #현재 문장이 다음 관광지에 대한 내용이니 변수 초기화 후 다시 추가
            
            else: #문장에서 어떤 관광지도 안 나온 경우, 계속 text 추가
                if i==0:
                    pass
                else:
                    place_texts = place_texts + text + '.'
            
    new_df.loc[new_df_idx + i]['날짜'] = df_test.iloc[idx]['날짜']
    new_df.loc[new_df_idx + i]['text'] = place_texts
    new_df.loc[new_df_idx + i]['index'] = idx
    new_df.loc[new_df_idx + i]['관광지'] = places[i]
    
    new_df_idx += len(places)

new_df

중문관광단지 : 중문관광단지 산방산부근 탄산온천이 있는 올레하우스는 우리의 5박을 책임질 숙소다자동차 방전으로 근처 식당에서 흑돼지요리로 첫외식을 했다.. 

산방산 : 중문관광단지 산방산부근 탄산온천이 있는 올레하우스는 우리의 5박을 책임질 숙소다자동차 방전으로 근처 식당에서 흑돼지요리로 첫외식을 했다.. 



,날짜,text,관광지,index
0,2021-01-01 07:35:00,중문관광단지 산방산부근 탄산온천이 있는 올레하우스는 우리의 5박을 책임질 숙소다자동...,중문관광단지,0
1,2021-01-01 07:35:00,,산방산,0
2,2021-01-01 07:35:00,색달해수욕장.우리의 첫 여행지다.해변을 걸었다..미국 씰비치에서 여름에 놀던 기억도...,색달해수욕장,0
3,2021-01-01 07:35:00,분위기로 알 것 같은 마음!용머리해안 마주보이는 산 멋진데.가짜 아우라 어깨에 매고...,용머리해안,0
4,2021-01-01 07:35:00,송악산사진 부탁하고 이모와 통화했다~~.늘 고맙고 좋아하고 존경하는 이모~~집으로 ...,송악산,0
...,...,...,...,...
13956,NaN,NaN,NaN,NaN
13957,NaN,NaN,NaN,NaN
13958,NaN,NaN,NaN,NaN
13959,NaN,NaN,NaN,NaN


In [12]:
new_df.loc[0]['text']

'중문관광단지 산방산부근 탄산온천이 있는 올레하우스는 우리의 5박을 책임질 숙소다자동차 방전으로 근처 식당에서 흑돼지요리로 첫외식을 했다..'

In [13]:
new_df.loc[1]['text']

''

In [14]:
new_df.loc[2]['text']

'색달해수욕장.우리의 첫 여행지다.해변을 걸었다..미국 씰비치에서 여름에 놀던 기억도 나고~~밑에서 위를 올려다보니 더 걷기 좋아보여서 장소를 옮겼다.눈에 정신이 팔려 작은 눈사람을 만들다 보조를 맞추기위해 포기하고 올라가는 중이다~~위엔 더 멋진 풍경과 .더 많은 눈들이 나를 기다리고 있었고~~올라프를 생각하며 미국식 눈사람을 만들었다..혹시나 싶어 준비한 라텍스 고무장갑 정말 유용하게 사용했다~~ㅎ늘 내 마음 속에 사랑과 고마움으로 가득찬 친구들에게 보내는 내마음을 담은 눈사람으로 사랑을 보냈다~~ㅎ다음장소는 .대포주상절리.코로나로 출입을 금지해서 매표를 하고 입장하는 곳은 갈 수 없었는데 주변 산책만으로도 충분히 멋지고 좋았다.동백꽃이 예쁘다~~우리 모자랑 깔맞춤 된 조개조형물~~.여긴 아프리카박물관고릴라 두마리고릴라 세마리 .내가 좋아하는 해바라기 뒤에 멀리 보이는 코끼리와 사자 조형물이 사파리에 와 있는 느낌이 들도록 찍은 사진인데~~.겨울이라 눈과 복장에 꿈깬 사진~~ㅋ장보기도 쉽게.홍당무들고 지나가던 아주머니 덕에 발견한~~필요한 식수는 필수.쌀도 필수.늘 밥상을 받고 살아서 .여기선 내가 주도적으로 상차리기를 하겠다 했다..누룽지밥은 시원한 숭늉은 따봉인데 밥심 필요한 내게는 힘이 딸려 쌀을 샀다~~.하멜상선전시관 근처 전경제주도 외국인들이 살고싶은 한국 1위.'

In [15]:
new_df.loc[3]['text']

'분위기로 알 것 같은 마음!용머리해안 마주보이는 산 멋진데.가짜 아우라 어깨에 매고 있는 부처가 자연스런 분위기 망쳤다는 느낌~~여기저기 코로나 마스크여기도 코로나로 출입금지 지역~~하멜씨 눈,비 맞으며 한국와서 계속 고생중~~봄에 이모와 함께 왔던 .'

In [16]:
new_df.loc[4]['text']

'송악산사진 부탁하고 이모와 통화했다~~.늘 고맙고 좋아하고 존경하는 이모~~집으로 들어와 맛있는 저녁 준비했다~~마트에서 공수해온 먹거리 .무 하나 사서 채나물 만들고.상진 좋아하는 달걀로 후라이~~그리고 숭늉으로 입가심.제주의 명품 물로 마무리~~~~♡♡♡아듀 2020~~^♡^.'

In [45]:
df_test.iloc[0]['내용'].split('\n')

['작년 결혼기념일(31주년)기념으로 같이 산 롱 패딩에',
 '32주년 기념 모자는 이번 여행 컨셉으로 내가 준비해 놨다.',
 '이번 여행은 미니멀을 추구한 남편과, 알뜰여행을 짝꿍 몰래 준비한 나와 좀 달랐지만 코로나가 준 경제적 손실을 고려, 귀엽게 바라봐 준 결과를 본셈이다~~ㅋ',
 '조기 여섯마리와 그것을 구울때 필요한 양파와 파는 얼려오고, 뭐라도 재료사서 해먹으려는 속셈이 고추가루와 소금 설탕 마늘까지 준비해 왔다.또 5박6일의 여행인지라 삼시세끼만 따져도 15번 이상의 식사를 외식만으로  할 수 없다는 생각이었다.',
 '거기다 난 외식을 즐기는 스타일도 아니고~~',
 '아침은 누릉지, 때론 토스트로 먹기로 했다.그래서 역곡에서 집으로 집에서 제주로 데려온 아삭아삭 양상추 ',
 '키플링 가방에 살짝 넣어왔다가 들켰다~~ㅋ',
 '어이없어하는 상진이 귀엽게 봐주는 눈치다~~ㅋ이 것들도 집에서부터 살짝 데리고 온 애들이다~~ㅎ',
 '중문관광단지 산방산부근 탄산온천이 있는 올레하우스는 우리의 5박을 책임질 숙소다자동차 방전으로 근처 식당에서 흑돼지요리로 첫외식을 했다.',
 '색달해수욕장',
 '우리의 첫 여행지다.해변을 걸었다.',
 '미국 씰비치에서 여름에 놀던 기억도 나고~~밑에서 위를 올려다보니 더 걷기 좋아보여서 장소를 옮겼다.눈에 정신이 팔려 작은 눈사람을 만들다 보조를 맞추기위해 포기하고 올라가는 중이다~~위엔 더 멋진 풍경과 ',
 '더 많은 눈들이 나를 기다리고 있었고~~올라프를 생각하며 미국식 눈사람을 만들었다.',
 '혹시나 싶어 준비한 라텍스 고무장갑 정말 유용하게 사용했다~~ㅎ늘 내 마음 속에 사랑과 고마움으로 가득찬 친구들에게 보내는 내마음을 담은 눈사람으로 사랑을 보냈다~~ㅎ다음장소는 ',
 '대포주상절리',
 '코로나로 출입을 금지해서 매표를 하고 입장하는 곳은 갈 수 없었는데 주변 산책만으로도 충분히 멋지고 좋았다.동백꽃이 예쁘다~~우리 모자랑 깔맞춤 된 조개조형물~~',
 '여긴 아프리카박물관고릴라 두마리고릴라

In [ ]:
#임시로 df_copy로 진행했으니 원본 df로 다시 해보기!